# PyOTA "Hello world" tutorial.

The goal of this tutorial is to create new empty wallet based on new random seed and to transfer IOTA tokens to it from wallet generated via faucet.

## Imports
Currently we have following imports:
 * iota - module name for [PyOTA](https://pyota.readthedocs.io/en/latest/)
 * secrets - "The [secrets](https://docs.python.org/3/library/secrets.html#module-secrets) module is used for generating cryptographically strong random numbers suitable for managing data such as passwords, account authentication, security tokens, and related secrets."
 * requests - "[Requests](http://docs.python-requests.org/en/master/) is an elegant and simple HTTP library for Python, built for human beings."

In [1]:
import iota
import secrets
import requests

# Generating part
This part is dedicated to generate receiver and sender seeds and addreses. If you run this code from top to bottom, new pairs will be generated every time. This might be not intended behaviour. If you like to use constant values while experimenting, re run your code from **Using part**.

## Generating and printing receiver seed
We can easily and securely generate seed which will be used as seed for receiver side.

In [2]:
chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ9'
receiver_seed = ''
for i in range(81): receiver_seed += secrets.choice(chars)
    
print(receiver_seed)

QVWIMXF9QVFJKWOBXAGEUFYLBJH9SUZDZUPSDABOFSVTL9FCSZCRFESPZNZB9QVTLBSQZVWNTUZPWERZW


## Connecting to testnet
Since we would like to be as close to actual Main IOTA network we will use IOTA testnet. IOTA testnet is supposed to be almost identical in operating as mainnet but has faster confirmation times and is reseted regurally, so tokens used there have no value. Here is list of example testnet nodes:

* http://p103.iotaledger.net:14700
* https://testnet140.tangle.works:443
* http://p101.iotaledger.net:14700

Please note that they could be down or unresponsive. 

In [70]:
uri = 'https://testnet140.tangle.works:443'

In [71]:
api = iota.Iota(uri, seed=receiver_seed)
api.get_node_info()

{'appName': 'IRI Testnet',
 'appVersion': '1.4.2.1',
 'duration': 0,
 'jreAvailableProcessors': 8,
 'jreFreeMemory': 330263224,
 'jreMaxMemory': 15271460864,
 'jreTotalMemory': 1800929280,
 'jreVersion': '1.8.0_161',
 'latestMilestone': TransactionHash(b'KBCCRBR99DBMPBQEIJC9NHDRJASYUYJXXUGKZK9ZYKYVTJXAFCTYIWSATZSUGIMXHL9BQG9FVNNDGI999'),
 'latestMilestoneIndex': 636075,
 'latestSolidSubtangleMilestone': TransactionHash(b'KBCCRBR99DBMPBQEIJC9NHDRJASYUYJXXUGKZK9ZYKYVTJXAFCTYIWSATZSUGIMXHL9BQG9FVNNDGI999'),
 'latestSolidSubtangleMilestoneIndex': 636075,
 'neighbors': 1,
 'packetsQueueSize': 0,
 'time': 1518786238107,
 'tips': 13,
 'transactionsToRequest': 13}

## Prepare receiver address
We already have seed for receiver (in variable **receiver_seed**), now we would like to have his address. Lets generate one:

In [4]:
gna_result = api.get_new_addresses(count=1)
addresses = gna_result['addresses']
receiver_address = addresses[0]
print("Receiver address 0 is: " + str(receiver_address))

Receiver address 0 is: DZMTOBWDQ9DZPWGJ9BOUSCOWKJDOFSJVCIXLLSEEQDRDUVOIY9QEKE9JLBNWAPLLFTELGATRGVIIV9QYW


## Prepare sender - how to get testnet tokens?
If we are generating new wallets from new seeds then thoose wallets will be empty (well, [probably](https://matthewwinstonjohnson.gitbooks.io/iota-guide-and-faq/how-secure-is-my-seed.html)). Since we would like to simulate transfer of "value" between two parties, we would need to have some tokens in at least one wallet in our setup. Thanks to IOTA devs, there is service called faucet which is genearting seeds and wallet addreses with some testnet tokens in it (2K at the moment). They also provide JSON API so we can use it right away.

In [5]:
r = requests.get('https://seeedy.tangle.works/')
print("Faucet response: " + str(r.status_code))
sender_wallet = r.json()

Faucet response: 200


In [39]:
sender_seed = sender_wallet["seed"]
sender_address = sender_wallet["address"]

In [40]:
print("Sender seed: " + sender_seed)
print("Sender address: " + sender_address)
print("Sender amount: " + str(sender_wallet["amount"]))

Sender seed: AESZMB9SVDQBWRQXSXOCDJVAZOVGMDCPMGHJIQASJJNGYYZUSSBKXPTBAIXSFFYBPQCNAUMLKNEGAHL9X
Sender address: ETQFDQUQHRMO9MXPEKCZMFWULGKOBZFCAUEQRSAFQTUOQIYZM9OJAGKMIWJIGURGMDDBONFIVEBOLGQ9C
Sender amount: 2000


# Using part
To this moment we should have seeds and addreses for sender (with tokens) and receiver. They are in:
* sender_seed
* sender_address
* receiver_seed
* receiver_address

In [51]:
api = iota.Iota(uri, seed=sender_seed)
sender_account = api.get_account_data(start=0)
sender_account["balance"]

2000

In [52]:
api = iota.Iota(uri, seed=receiver_seed)
receiver_account = api.get_account_data(start=0)
receiver_account["balance"]

0

## Prepare transaction

Lets switch back to sender seed and prapare message. Such message is build from string and will be attached to transaction, so receiver will be able to easily see it.

In [10]:
api = iota.Iota(uri, seed=sender_wallet["seed"])

In [53]:
message = ("Here, have all my testnet tokens!")

Proposed transaction must consist receiver **address**, **value** we would like to send and **message** which will be added to transfer.

In [75]:
proposedTransaction = iota.ProposedTransaction(address = iota.Address(receiver_address), 
                                               value = sender_wallet["amount"], 
                                               message = iota.TryteString.from_string(message)
                                              )

In [76]:
proposedTransaction

ProposedTransaction(**{'address': Address(b'DZMTOBWDQ9DZPWGJ9BOUSCOWKJDOFSJVCIXLLSEEQDRDUVOIY9QEKE9JLBNWAPLLFTELGATRGVIIV9QYW'),
                     'attachment_timestamp': 0,
                     'attachment_timestamp_lower_bound': 0,
                     'attachment_timestamp_upper_bound': 0,
                     'branch_transaction_hash': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
                     'bundle_hash': None,
                     'current_index': None,
                     'hash_': None,
                     'last_index': None,
                     'legacy_tag': Tag(b'999999999999999999999999999'),
                     'nonce': Nonce(b'999999999999999999999999999'),
                     'signature_message_fragment': None,
                     'tag': Tag(b'999999999999999999999999999'),
                     'timestamp': 1518790597,
                     'trunk_transaction_hash': TransactionHash(b'999999999999999

# Execute transfer

In [77]:
depth = 3

In [78]:
transfer = api.send_transfer(transfers = [proposedTransaction], 
                  depth = depth,
                  inputs = [iota.Address(sender_wallet["address"], key_index=0, security_level=2)]
                 ) # should work on working and synced node

BadApiResponse: 400 response from node: The subtangle is not solid

In [55]:
transactionHash = []
for transaction in transfer["bundle"]:
    transactionHash.append(transaction.hash)
    print(transaction.address, transaction.hash)

DZMTOBWDQ9DZPWGJ9BOUSCOWKJDOFSJVCIXLLSEEQDRDUVOIY9QEKE9JLBNWAPLLFTELGATRGVIIV9QYW SLVJRIMDZETOICVGJXRFKBJHKYAZGSTSRUVVLQYDE9XUKTQBQVNDFAOMXNKELSSTGKDIXSWKORNMA9999
ETQFDQUQHRMO9MXPEKCZMFWULGKOBZFCAUEQRSAFQTUOQIYZM9OJAGKMIWJIGURGMDDBONFIVEBOLGQ9C GEEQWJEZOJEPIAIOIATSLVKMDZOJWHNTFQNRACEMZMDDWAQAQXRMCU9NAAMLRMAIHK9FKWEBEQRFZ9999
ETQFDQUQHRMO9MXPEKCZMFWULGKOBZFCAUEQRSAFQTUOQIYZM9OJAGKMIWJIGURGMDDBONFIVEBOLGQ9C TCOZUFHESRRAFWRWYT9NHWPFYXKLOLRFHQEWMZKFKHJXRQ9XVOPGCPTBGCCBMLJXZYQGMGTMIXVWZ9999


Lets check statuses of our transactions. We need hashes of them to be able to check if they were executed.

In [59]:
api.get_latest_inclusion(transactionHash)

{'states': {TransactionHash(b'GEEQWJEZOJEPIAIOIATSLVKMDZOJWHNTFQNRACEMZMDDWAQAQXRMCU9NAAMLRMAIHK9FKWEBEQRFZ9999'): False,
  TransactionHash(b'SLVJRIMDZETOICVGJXRFKBJHKYAZGSTSRUVVLQYDE9XUKTQBQVNDFAOMXNKELSSTGKDIXSWKORNMA9999'): False,
  TransactionHash(b'TCOZUFHESRRAFWRWYT9NHWPFYXKLOLRFHQEWMZKFKHJXRQ9XVOPGCPTBGCCBMLJXZYQGMGTMIXVWZ9999'): False}}

If the status is "True", it means that transaction was executed and we could see that tokens appeared on receiver side.
But if we have only "False" that means that our proposed transaction wasnt validated and included in tangle.
In this case we could wait or replay our request.

In [65]:
api.replay_bundle(transactionHash[0], depth=depth) # should work on working and synced node

BadApiResponse: 400 response from node: The subtangle is not solid

After succesfull inclusion, we should be able to see transfered tokens in receiver wallet:

In [72]:
api = iota.Iota(uri, seed=sender_seed)
sender_account = api.get_account_data(start=0)
sender_account["balance"]

2000

In [73]:
api = iota.Iota(uri, seed=receiver_seed)
receiver_account = api.get_account_data(start=0)
receiver_account["balance"]

0

## Versions of modules used in tutorial:

In [12]:
assert iota.__version__ == '2.0.4'
assert requests.__version__ == '2.18.4'

## Future enchancements:

### Check if address was used to spent: https://github.com/iotaledger/iota.lib.py/pull/158
Not pushed to relese yet.

In [79]:
api = Iota(uri, seed=sender_seed)
r = api.were_addresses_spent_from(sender_address)
print(r)

InvalidCommand: Iota does not support 'were_addresses_spent_from' command.